<a href="https://colab.research.google.com/github/hanghae-plus-AI/AI-1-soyoungcareer/blob/main/week3/Chapter3_1_%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 과제 수행 순서
    1. AG_News dataset 준비
        1. `fancyzhx/ag_news` dataset load
        2. Truncation 관련 제거
    2. Classifier output, loss function, accuracy function 변경
        1. `nn.CrossEntropyLoss` 추가
        2. `TextClassifier`의 출력 차원 조정
        3. `accuracy` 함수 수정
    3. attention_mask 처리



- 질문사항
  1. Train Loss가 줄어드는 듯 하다가 줄지 않는 이유를 모르겠음.

# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 이전 주차 실습에서 사용하던 감정 분석 문제에 적용합니다. 먼저 필요한 library들을 불러옵니다.

In [1]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00


그 후, 우리가 사용하는 DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://h

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


DistilBERT의 tokenizer를 불러왔으면 이제 `collate_fn`과 data loader를 정의합니다. 이 과정은 이전 실습과 동일하게 다음과 같이 구현할 수 있습니다.

In [3]:
# ds = load_dataset("stanfordnlp/imdb")
# 1-a. AG_News dataset 준비
ds = load_dataset("fancyzhx/ag_news")

# 라벨의 고유 값 확인
train_labels = ds['train']['label']
unique_labels = set(train_labels)

print(f"Number of unique labels: {len(unique_labels)}")
print(f"Unique labels: {unique_labels}")

for example in list(ds['train'])[:5]:
    print(f"Text: {example['text']}\nLabel: {example['label']}")


def collate_fn(batch):
  max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  # texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  # 1-b. truncation 제거 : 뉴스 기사이기 때문에 임의로 데이터를 자르면 중요한 정보가 잘릴 가능성이 있기 때문에 입력 길이를 제한하지 않고 모든 문장을 padding만 해줌.
  # texts = torch.LongTensor(tokenizer(texts, padding=True).input_ids)

  # 3. input_ids와 함께 attention_mask도 받음
  encodings = tokenizer(texts, padding=True, return_tensors="pt")    # return_tensors="pt" : tokenizer가 숫자로 변환한 데이터를 PyTorch 텐서로 반환하도록 해 주는 옵션
  texts = encodings.input_ids   # 숫자로 변환된 문장
  attention_mask = encodings.attention_mask    # 어디가 진짜 텍스트이고, 어디가 패딩된 빈칸인지를 알려주는 마스크 정보
  labels = torch.LongTensor(labels)

  # print(f"encodings: {encodings}")
  # print(f"texts: {texts}")
  # print(f"attention_mask: {attention_mask}")
  # print(f"labels: {labels}")

  return texts, attention_mask, labels

batch_size = 64

train_loader = DataLoader(
    ds['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Number of unique labels: 4
Unique labels: {0, 1, 2, 3}
Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
Label: 2
Text: Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
Label: 2
Text: Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
Label: 2
Text: Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Sa

collate_fn 함수에서

    print(f"encodings: {encodings}")
    print(f"texts: {texts}")
    print(f"attention_mask: {attention_mask}")
    print(f"labels: {labels}")

프린트 해봤을 때의 결과
=>

    encodings: {'input_ids': tensor([[  101,  7328,  3065,  ...,     0,     0,     0],
            [  101, 15662,  2879,  ...,     0,     0,     0],
            [  101, 17953, 10236,  ...,     0,     0,     0],
            ...,
            [  101,  6261,  2177,  ...,     0,     0,     0],
            [  101,  3097,  3343,  ...,     0,     0,     0],
            [  101,  3623,  4037,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            ...,
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0]])}
    texts: tensor([[  101,  7328,  3065,  ...,     0,     0,     0],
            [  101, 15662,  2879,  ...,     0,     0,     0],
            [  101, 17953, 10236,  ...,     0,     0,     0],
            ...,
            [  101,  6261,  2177,  ...,     0,     0,     0],
            [  101,  3097,  3343,  ...,     0,     0,     0],
            [  101,  3623,  4037,  ...,     0,     0,     0]])
    attention_mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            ...,
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0]])
    labels: tensor([3, 1, 1, 1, 3, 0, 0, 0, 2, 3, 2, 2, 2, 3, 0, 1, 1, 2, 2, 1, 1, 3, 1, 0,
            2, 0, 3, 1, 1, 0, 1, 0, 1, 3, 2, 3, 0, 1, 2, 3, 0, 1, 1, 2, 2, 1, 3, 1,
            3, 2, 3, 2, 1, 0, 3, 2, 0, 1, 1, 2, 3, 3, 0, 3])
    encodings: {'input_ids': tensor([[  101,  1012,  6097,  ...,     0,     0,     0],
            [  101,  2678,  2399,  ...,     0,     0,     0],
            [  101,  3664,  4491,  ...,     0,     0,     0],
            ...,
            [  101,  1057,  1012,  ...,     0,     0,     0],
            [  101,  7513,  2758,  ...,     0,     0,     0],
            [  101,  5522, 15768,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            ...,
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0]])}
    texts: tensor([[  101,  1012,  6097,  ...,     0,     0,     0],
            [  101,  2678,  2399,  ...,     0,     0,     0],
            [  101,  3664,  4491,  ...,     0,     0,     0],
            ...,
            [  101,  1057,  1012,  ...,     0,     0,     0],
            [  101,  7513,  2758,  ...,     0,     0,     0],
            [  101,  5522, 15768,  ...,     0,     0,     0]])
    attention_mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            ...,
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0]])

이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

In [4]:
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [5]:
from torch import nn


class TextClassifier(nn.Module):
  def __init__(self):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.dropout = nn.Dropout(self.encoder.config.seq_classif_dropout)
    # 과적합 방지
    # seq_classif_dropout (float, optional, defaults to 0.2) – The dropout probabilities used in the sequence classification and the multiple choice model DistilBertForSequenceClassification.
    # https://huggingface.co/transformers/v3.5.1/model_doc/distilbert.html?highlight=seq_classif_dropout

    # 2-b. 출력차원 조정 : AG News 데이터셋은 4개의 뉴스 카테고리(World, Sports, Business, Sci/Tech)로 나뉨.
    self.classifier = nn.Linear(768, 4)

  def forward(self, input_ids, attention_mask):
    # x = self.encoder(x)['last_hidden_state']

    # attention_mask를 encoder에 전달
    x = self.encoder(input_ids, attention_mask=attention_mask)['last_hidden_state']
    x = self.dropout(x[:, 0])   # 드롭아웃 적용
    # [CLS] token의 representation만 사용
    x = self.classifier(x)

    return x


model = TextClassifier()

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [6]:
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [7]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
# loss_fn = nn.BCEWithLogitsLoss()
loss_fn = nn.CrossEntropyLoss()    # 2-a. nn.CrossEntropyLoss 추가

optimizer = Adam(model.parameters(), lr=lr)
# scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # 학습률 스케줄러: 3에포크마다 학습률을 0.1배로 줄임
n_epochs = 10

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    # 3. attention_mask 처리
    inputs, attention_mask, labels = data
    # inputs, labels = inputs.to('cuda'), labels.to('cuda').float()
    # float() 제거 : 다중 분류에서는 일반적으로 정수 타입으로 사용함. CrossEntropyLoss는 정수형 라벨을 사용함.
    inputs, attention_mask, labels = (
        inputs.to('cuda'),
        attention_mask.to('cuda'),
        labels.to('cuda')
    )

    # preds = model(inputs)[..., 0]
    preds = model(inputs, attention_mask)
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    # print("preds.shape : ", preds.shape)
    # print("labels.shape : ", labels.shape)

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

Epoch   0 | Train Loss: 690.8601857051253
Epoch   1 | Train Loss: 587.4298562556505
Epoch   2 | Train Loss: 579.1556745842099
Epoch   3 | Train Loss: 575.8851155489683
Epoch   4 | Train Loss: 575.0558534041047
Epoch   5 | Train Loss: 573.578078225255
Epoch   6 | Train Loss: 572.8358936160803
Epoch   7 | Train Loss: 569.8192961290479
Epoch   8 | Train Loss: 572.9546413943172
Epoch   9 | Train Loss: 572.1323872208595


**=> Train Loss 가 줄어들지 않는 이유가 뭔지 모르겠음.**

shape 확인 결과

    preds.shape :  torch.Size([64, 4])
    labels.shape :  torch.Size([64])

In [8]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    # 3. attention_mask 처리
    inputs, attention_mask, labels = data
    inputs, attention_mask, labels = (
        inputs.to('cuda'),
        attention_mask.to('cuda'),
        labels.to('cuda')
    )

    preds = model(inputs, attention_mask)
    # 2-c. 다중분류 차원 맞춰줌.
    preds = torch.argmax(preds, dim=-1)
    # preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")


=========> Train acc: 0.902 | Test acc: 0.900


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.